In [1]:
import cadquery as cq
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show)
from jupyter_cadquery import set_sidecar, set_defaults

import alphashape as alph

import numpy as np
from scipy.ndimage import affine_transform

import geojson 

import makoreactor as mr

set_defaults(default_color="#92baff", alpha=0.5, black_edges=True, axes=False, pinning=True, grid=False, theme="dark", axes0=True, ortho=True, transparent=True)
set_sidecar("CadQuery", init=True)
    
in2mm = lambda x: x*25.4

lzip = lambda *args, **kwargs: list(zip(*args, **kwargs))

def get_poly_verts(poly):
    verts = poly.vertices().vals()
    coords_x = [v.X for v in verts]
    coords_y = [v.Y for v in verts]
    return coords_x, coords_y


Overwriting auto display for cadquery Workplane and Shape
Paramater alpha is not a valid argument for show()
Paramater pinning is not a valid argument for show()


/home/otacon/anaconda3/envs/cadquery/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [2]:
from affine import Affine

mr.fgcF1CapLayout.layout_affine = Affine.translation(0, -13)
fgc = mr.f1CapMako1.set(width=in2mm(11.8), hole_deltax=300, n_modelu=2, height=in2mm(5), layout=mr.fgcF1CapLayout)
fgc.generate()

[-20.  20.]
BRep_API: command not done
Done, using side car 'Cadquery'


In [6]:
switchplate = fgc.parts[2].generate()

In [7]:
brook = cq.Workplane("XY").rect(100,50).translate((-45,0)).extrude(10, both=True).translate((-75,-25))
final_parts = [
    fgc.parts[0].generate(),
    fgc.parts[1].generate(),
    fgc.parts[2].generate().cut(brook),
    fgc.parts[3].generate().cut(brook),
    fgc.parts[4].generate()
]

show(*final_parts)

Done, using side car 'Cadquery'


In [6]:
!mkdir mako1_fgc 

In [7]:
from cadquery import exporters

for i,p in enumerate(final_parts):
    exporters.export(p.edges(">Z"), f"mako1_fgc/layer{i}.dxf")

In [2]:
text = (cq.Workplane("XY")
         .text("MAKO", 70, 10, kind='italic',halign='center', valign='center',fontPath='/mnt/chromeos/MyFiles/Downloads/Oswald-Bold.ttf')
         .rotate((0, 0, 1), (0, 0, 0), 90)
         .translate((-(faceplate.width/2)+40, 5, -2.5))
)
text = text.edges("|Z").fillet(1)
# text.

show(faceplate_part.union(text).edges(">Z or >>Z[0]").edges())

NameError: name 'faceplate' is not defined

In [5]:
text = (cq.Workplane("XY")
         .text("MAKO", 130, 10,halign='center', valign='center',fontPath='/mnt/chromeos/MyFiles/Downloads/Oswald-Bold.ttf')
)
text = text.edges("|Z").fillet(1)
show(faceplate_part, text)

Done, using side car 'Cadquery'


In [29]:
faceplate

(KeycapFaceplate(width=355.59999999999997, height=152.39999999999998, depth=3.175, hole_diameter=10, hole_offset=25, hole_deltax=275, hole_deltay=275, fillet_radius=8, keycap_dimensions=(20.5, 20.5), keycap_fillet_radius=0.1, layout=<makogen.layouts.CircleCapLeverless object at 0x7916d7b78bb0>),)

In [57]:
val = mg.KeycapLayout.geom.extrude(5)
val

Done, using side car 'Cadquery'


In [58]:
pico = cq.Workplane("XY").rect(21, 51).extrude(1).rect(11.4, 47, forConstruction=True).vertices().circle(2.1/2).cutThruAll()

In [59]:
pico

Done, using side car 'Cadquery'


In [426]:
dimensions = (21, 52.3, 3.6)

pitch = 2.54
pitch_wall_thickness = 0.5

pcb_thickness = 1

pcb_offset_x = 5
pcb_offset_y = 10
pcb_offset_z = 0
peg_len = 2.5

wall_thickness = 2
bottom_thickness = .5

width = dimensions[0] + pcb_offset_x + wall_thickness
length = dimensions[1] + pcb_offset_y
depth  = dimensions[2] + peg_len + (bottom_thickness) + pcb_offset_z 

pico_holder = cq.Workplane("XY").box(width, length, depth)
cavity = (cq.Workplane("XY")
          .box(width-(wall_thickness*2), 
               length, 
               depth).translate((0, 0, bottom_thickness))
)

pico_holder = pico_holder.cut(cavity).translate((0,0,depth/2))

pico_holder = (
    pico_holder
# .edges("|Z and (<<Y[2] or >>Y[2])")
.edges("|Y and <<Z[-2]").fillet(2)
)

# create wire holes
wire_centers = np.arange(-dimensions[1]/2, dimensions[1]/2, pitch)
wire_verts = list(zip([0]*len(wire_centers), wire_centers))

wire_side_gaps = (cq.Workplane("XY")
                  .pushPoints(wire_verts)
                  .rect(width+10, pitch-pitch_wall_thickness)
                  .extrude(dimensions[2])
                  )

wire_side_gaps = wire_side_gaps.translate((0, 0, bottom_thickness))
# wire_side_gaps = (
#     cq.Workplane("XY")
#     .box(width, dimensions[1], dimensions[2])
#     .translate((0,0,(depth/4) + bottom_thickness*2))
# )

pico_holder = pico_holder.cut(wire_side_gaps)

show(pico_holder)

Done, using side car 'Cadquery'


In [427]:
depth

6.6

In [428]:
# pico_holder = cq.Workplane("XY").box(dimensions[0], dimensions[1], bottom_thickness).edges("|Z").fillet(4)
feet = (cq.Workplane("XY").rect(11.4, 47, forConstruction=True)
        .vertices().circle(1.75).extrude(dimensions[2]+pcb_offset_z-pcb_thickness)
        .faces(">Z").circle(.9).extrude(peg_len)
        .translate((0,0,bottom_thickness))
       )

pico_holder.add(feet)

Done, using side car 'Cadquery'


In [387]:
from cadquery import exporters
exporters.export(pico_holder, "artifacts/pico_holder.stl")

In [8]:
from affine import Affine
from dataclasses import dataclass

    
one_thumb_fgc = mg.FgcLeverless()

(cq.Workplane("XY")
 .rect(mg.in2mm(13.5), mg.in2mm(5.75))
 .extrude(5).edges("|Z")
 .fillet(10)
 .cut(one_thumb_fgc.geom.extrude(5)) 
)

called
Done, using side car 'Cadquery'


Done, using side car 'Cadquery'


In [7]:
mg.fgcLayout
mg.mako1.set(width=in2mm(12), height=in2mm(5), layout=mg.fgcF1CapLayout).generate()

Done, using side car 'Cadquery'


In [ ]:
mako1.parts[2].layout

In [6]:
from affine import Affine

mg.OneThumbFgcLayout.misc_affine = Affine.translation(-95, 57)

In [4]:

b = mg.Base(depth=in2mm(1/8), hole_diameter=10, height=in2mm(8)).create()
r = cq.Workplane("XY").pushPoints(mg.OneThumbFgcLayout.layout).rect(14,14).extrude(10, both=True)
b.cut(r)

Done, using side car 'Cadquery'


In [4]:
# import json
# j = str(assembly.dump())
# j.replace(" ", "")

In [10]:
from dataclasses import asdict
str(mg.F1CapLayout)

'CircleCapLeverless'

In [ ]:
import zlib

a = "this string needs compressing"
a = zlib.compress(a.encode())
print(zlib.decompress(a).decode())  # outputs original contents of a

In [15]:
from dataclasses import asdict
asdict?

Signature: asdict(obj, *, dict_factory=<class 'dict'>)
Docstring:
Return the fields of a dataclass instance as a new dictionary mapping
field names to field values.

Example usage:

  @dataclass
  class C:
      x: int
      y: int

  c = C(1, 2)
  assert asdict(c) == {'x': 1, 'y': 2}

If given, 'dict_factory' will be used instead of built-in dict.
The function applies recursively to field values that are
dataclass instances. This will also look into built-in containers:
tuples, lists, and dicts.
File:      ~/anaconda3/envs/cadquery/lib/python3.9/dataclasses.py
Type:      function


In [19]:
# len(s), len(assembly.dumps())

In [20]:
# zlib.decompress(bytes.fromhex(s))

In [21]:
# cardboardmx = PartGroup(
#     [
#         # Part(cq.importers.importsvg('/mnt/chromeos/MyFiles/Downloads/3_SwitchPlate_v2.svg').rotateAboutCenter((0,0,1), 180).translate((223.1, -181.25, 0)) ),
#         Part(base, "bottom"),
#         # Part(wiring_cavity.translate((0,0,depthmm*1)), "pcb_holder2"),
#         Part(wiring_cavity.translate((0,0,depthmm*1)), "pcb_holder", color="black"),
#         Part(switch_plate.translate((0,0,depthmm*2)), "switchplate2", color="black"),
#         Part(f1cap_faceplate.translate((0,0,depthmm*3)), "faceplate1"),
#         # Part(dowel.translate((widthmm/2-dowel_width/4,heightmm/2-dowel_offset/2, 10)), "dowel")

#     ],
#     "carboardMX"
# )

# show(cardboardmx, default_color="blue")

In [23]:
opts = {
    "projectionDir": (0, 0, 0.5),
}

In [3]:

# exporters.export(dowel.edges(">Z"), "artifacts/dowel.dxf")

In [28]:
points

[[10, 30],
 [15.0, 20.0],
 [15.0, 0.0],
 [10, -10],
 [5.0, 0.0],
 [5.0, 20.0],
 [10, 30]]